In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
storm_df = pd.read_csv("../data/storm-df.csv")

C:\Users\robsc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17,26,27,29,30,35,36,38,40,41,43,44,49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
storm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598425 entries, 0 to 1598424
Data columns (total 52 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Unnamed: 0          1598425 non-null  int64  
 1   BEGIN_YEARMONTH     1598425 non-null  int64  
 2   BEGIN_DAY           1598425 non-null  int64  
 3   BEGIN_TIME          1598425 non-null  int64  
 4   END_YEARMONTH       1598425 non-null  int64  
 5   END_DAY             1598425 non-null  int64  
 6   END_TIME            1598425 non-null  int64  
 7   EPISODE_ID          1491651 non-null  float64
 8   EVENT_ID            1598425 non-null  int64  
 9   STATE               1598424 non-null  object 
 10  STATE_FIPS          1598424 non-null  float64
 11  YEAR                1598425 non-null  int64  
 12  MONTH_NAME          1598425 non-null  object 
 13  EVENT_TYPE          1598425 non-null  object 
 14  CZ_TYPE             1598425 non-null  object 
 15  CZ_FIPS        

In [4]:
event_count = storm_df.groupby(['STATE'])['EVENT_TYPE'].count().sort_values(ascending=False)

In [5]:
event_count_df = pd.DataFrame(event_count).reset_index()

In [6]:
event_count_df
event_count_df.head()

,STATE,EVENT_COUNT
0,TEXAS,122229
1,KANSAS,74458
2,OKLAHOMA,59906
3,MISSOURI,56676
4,IOWA,56411


In [7]:
state_size_df = pd.read_csv('../data/state_size_df.csv', dtype={'SQ_MILE': 'Int64'})

In [8]:
merged = pd.merge(storm_df, state_size_df, on='STATE', how='left')

In [9]:
merged = pd.merge(merged, event_count_df, on='STATE', how='left')

In [10]:
merged['EVENT_COUNT']= merged['EVENT_COUNT'].astype('Int64')

In [11]:
merged['EVENT_PER_SQ_MILE'] = round(merged['EVENT_COUNT']/merged['SQ_MILE'],2)

In [13]:
merged.groupby(['STATE'])['EVENT_PER_SQ_MILE'].mean().sort_values(ascending=False)

STATE
NEW JERSEY              3.18
HAWAII                  2.19
MARYLAND                2.14
DELAWARE                2.14
RHODE ISLAND             2.1
MASSACHUSETTS           1.81
CONNECTICUT             1.21
VIRGINIA                 1.2
KENTUCKY                1.12
IOWA                    1.01
OHIO                    1.01
VERMONT                 0.98
NEW YORK                0.97
PENNSYLVANIA            0.97
INDIANA                 0.94
WEST VIRGINIA           0.93
NORTH CAROLINA          0.92
KANSAS                  0.91
ILLINOIS                 0.9
SOUTH CAROLINA          0.88
TENNESSEE               0.87
OKLAHOMA                0.87
MISSOURI                0.82
NEW HAMPSHIRE            0.8
GEORGIA                 0.78
ARKANSAS                0.74
WISCONSIN               0.68
MISSISSIPPI             0.67
ALABAMA                 0.65
NEBRASKA                0.63
FLORIDA                 0.58
SOUTH DAKOTA            0.56
MINNESOTA               0.55
LOUISIANA               0.53
MICHIGAN

In [14]:
merged_new = pd.DataFrame(merged, columns=['STATE', 'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_NAME', 'BEGIN_DATE_TIME', 'DEATHS_DIRECT', 'TOR_F_SCALE', 'SQ_MILE', 'EVENT_COUNT', 'EVENT_PER_SQ_MILE'])
merged_new

,STATE,YEAR,MONTH_NAME,EVENT_TYPE,CZ_NAME,BEGIN_DATE_TIME,DEATHS_DIRECT,TOR_F_SCALE,SQ_MILE,EVENT_COUNT,EVENT_PER_SQ_MILE
0,COLORADO,1987,May,Thunderstorm Wind,PHILLIPS,02-MAY-87 18:25:00,0,NaN,103718,40529,0.39
1,COLORADO,1987,May,Tornado,DENVER,12-MAY-87 17:05:00,0,F1,103718,40529,0.39
2,COLORADO,1987,May,Thunderstorm Wind,DOUGLAS,17-MAY-87 15:49:00,0,NaN,103718,40529,0.39
3,COLORADO,1987,May,Hail,WASHINGTON,17-MAY-87 17:55:00,0,NaN,103718,40529,0.39
4,COLORADO,1987,May,Thunderstorm Wind,WASHINGTON,17-MAY-87 18:18:00,0,NaN,103718,40529,0.39
...,...,...,...,...,...,...,...,...,...,...,...
1598420,NEW MEXICO,2021,December,Drought,UPLANDS OF THE BOOTHEEL,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598421,NEW MEXICO,2021,December,Drought,SOUTHWEST DESERT/MIMBRES BASIN,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598422,NEW MEXICO,2021,December,Drought,EASTERN BLACK RANGE FOOTHILLS,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598423,NEW MEXICO,2021,December,Drought,SIERRA COUNTY LAKES,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15


In [15]:
world_df = pd.read_csv('../data/world-df.csv')

In [16]:
world_df.columns

Index(['Unnamed: 0', 'YEAR-MONTH', 'LONGITUDE (deg)', 'LATITUDE (deg)', 'MEAN SEA LEVEL PRESSURE (0.1 mb)', 'VAPOR PRESSURE (0.1 mb)', 'MEAN TEMPERATURE (deg C)', 'NO. DAYS PRECIPITATION >= 1.0 mm', 'TOTAL PRECIPITATION (mm)', 'SUNSHINE DURATION (hours)', 'STATION NAME', 'YEAR', 'MONTH', 'MEAN FAHRENHEIT'], dtype='object')

In [17]:
world_df = pd.DataFrame(world_df)

In [18]:
world_df = world_df.replace({-9999:  None})

In [ ]:
world_df[['YEAR','MONTH']] = world_df['YEAR-MONTH'].apply(
   lambda x: pd.Series(str(x).split("-")))

In [ ]:
world_df['MEAN FAHRENHEIT'] = ((world_df['MEAN TEMPERATURE (deg C)']*1.8)/10 + 32)
world_df = world_df[world_df['STATION NAME'].notnull()]
#world_df1.info()

In [ ]:
world_df1['YEAR'] = world_df1['YEAR'].astype('float64')

In [ ]:
world_df1['MONTH'] = world_df1['MONTH'].astype('float64')

In [ ]:
merged_new['YEAR'] = merged_new['YEAR'].astype('float64')

In [20]:
world_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844814 entries, 0 to 844813
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Unnamed: 0                        844814 non-null  int64  
 1   YEAR-MONTH                        844814 non-null  object 
 2   LONGITUDE (deg)                   844814 non-null  float64
 3   LATITUDE (deg)                    844814 non-null  float64
 4   MEAN SEA LEVEL PRESSURE (0.1 mb)  764981 non-null  float64
 5   VAPOR PRESSURE (0.1 mb)           815379 non-null  float64
 6   MEAN TEMPERATURE (deg C)          831586 non-null  float64
 7   NO. DAYS PRECIPITATION >= 1.0 mm  808959 non-null  float64
 8   TOTAL PRECIPITATION (mm)          810188 non-null  float64
 9   SUNSHINE DURATION (hours)         513361 non-null  float64
 10  STATION NAME                      844814 non-null  object 
 11  YEAR                              844814 non-null  f

In [ ]:
world_df.to_csv('../data/world-df.csv')

In [24]:
merged_new.to_csv('../data/storm-df-final.csv')

In [22]:
world_df.groupby(['STATION NAME'])['MEAN FAHRENHEIT'].mean().sort_values(ascending=False)

STATION NAME
JAHRA                             100.94
SALMY                              96.44
BUBIAN ISLAND                      96.26
SOUTH DOLPHIN                      92.84
KASHAN                             91.22
                                   ...  
SAO GABRIEL DA CACHOEIRA (AERO       NaN
SCONE SCS                            NaN
TIRIOS                               NaN
TOKUA W.O.                           NaN
WASA EP5                             NaN
Name: MEAN FAHRENHEIT, Length: 3772, dtype: float64

In [23]:
merged_new

,STATE,YEAR,MONTH_NAME,EVENT_TYPE,CZ_NAME,BEGIN_DATE_TIME,DEATHS_DIRECT,TOR_F_SCALE,SQ_MILE,EVENT_COUNT,EVENT_PER_SQ_MILE
0,COLORADO,1987,May,Thunderstorm Wind,PHILLIPS,02-MAY-87 18:25:00,0,NaN,103718,40529,0.39
1,COLORADO,1987,May,Tornado,DENVER,12-MAY-87 17:05:00,0,F1,103718,40529,0.39
2,COLORADO,1987,May,Thunderstorm Wind,DOUGLAS,17-MAY-87 15:49:00,0,NaN,103718,40529,0.39
3,COLORADO,1987,May,Hail,WASHINGTON,17-MAY-87 17:55:00,0,NaN,103718,40529,0.39
4,COLORADO,1987,May,Thunderstorm Wind,WASHINGTON,17-MAY-87 18:18:00,0,NaN,103718,40529,0.39
...,...,...,...,...,...,...,...,...,...,...,...
1598420,NEW MEXICO,2021,December,Drought,UPLANDS OF THE BOOTHEEL,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598421,NEW MEXICO,2021,December,Drought,SOUTHWEST DESERT/MIMBRES BASIN,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598422,NEW MEXICO,2021,December,Drought,EASTERN BLACK RANGE FOOTHILLS,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
1598423,NEW MEXICO,2021,December,Drought,SIERRA COUNTY LAKES,01-DEC-21 00:00:00,0,NaN,121356,18478,0.15
